In [1]:
import pandas as pd
import pickle
from Bio import SeqIO
import os
import numpy as np
import subprocess
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

## Loading testing dataframe and removing MIBiG BGCs

In [2]:
training_df = pd.read_csv("./outputs/affinity_df-round4-TFL210427.txt",index_col=0)

training_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspCM7v1,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF995
5677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996
5678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996
5679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF997


In [3]:
training_bgcs = pickle.load( open("./outputs/affinity_bgcs-round4-TFL210427.txt", "rb" ) )

training_bgcs

['GCA_000737335.region031',
 'GCA_002994635.region018',
 'GCA_000170895.region002',
 'GCA_002994615.region022',
 'BGC0000844.1',
 'DS570624.region002',
 'BGC0000845.1',
 'BGC0000319.1',
 'DS570624.region019',
 'BGC0000373.1',
 'BGC0000940.1',
 '2747842505.region030',
 'GCA_002899455.region014',
 '2517572165.region052',
 'GCA_000365385.region030',
 'GCA_000377145.region006',
 'GCA_000424965.region016',
 'GCA_000377525.region018',
 'GCA_000514715.region025',
 '651717009.region006',
 'DS570624.region031',
 'GCA_000426165.region008',
 '2518285564.region046',
 'GCA_000158915.region019',
 'GCA_000377165.region004',
 'GCA_000377125.region004',
 '2515154206.region008',
 'GCA_000739105.region018',
 'GCA_000527195.region007',
 'BGC0000941.1',
 'GCA_000156695.region008',
 'GCA_000424765.region022',
 'GCA_000365385.region010',
 'GCA_000424805.region006',
 'GCA_000240165.region027',
 'GCA_000515055.region002',
 'GCA_000156435.region015',
 'BGC0001478.1',
 'GCA_000203835.region011',
 'GCA_000701285.

In [4]:
training_df['bgc'] = training_bgcs

training_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label,bgc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000737335.region031
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994635.region018
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000170895.region002
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994615.region022
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF2,BGC0000844.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF995,ERR3946851_contigs.region013
5677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3941157_contigs.region006
5678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3946770_contigs.region007
5679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF997,ERR3946819_contigs.region008


In [5]:
index_to_keep = []

for i,r in training_df.iterrows():
    if 'BGC' not in r['bgc']:
        index_to_keep.append(i)
        
training_df = training_df.loc[index_to_keep]
training_df.reset_index(inplace=True,drop=True)

training_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label,bgc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000737335.region031
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994635.region018
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000170895.region002
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994615.region022
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF2,DS570624.region002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF995,ERR3946851_contigs.region013
5417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3941157_contigs.region006
5418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3946770_contigs.region007
5419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF997,ERR3946819_contigs.region008


## Annotating BGC class

In [6]:
names_dict = {}
with open("./outputs/names_dict-round4-TFL210723.txt") as f:
    for line in f:
        (key, val) = line.split(',')
        names_dict[key] = val.strip('\n')
        
names_dict

{'c00017_NODE_17...region001': 'ERR2239797_contigs.region003',
 'c00219_NODE_21...region001': 'ERR2239719_contigs.region004',
 'c00533_NODE_53...region001': 'ERR3946850_contigs.region002',
 'c00003_NODE_3_...region001': 'ERR2239707_contigs.region002',
 'c00013_NODE_13...region001': 'ERR2239715_contigs.region001',
 'c00024_NODE_24...region001': 'ERR2239791_contigs.region002',
 'c00045_NODE_45...region001': 'ERR2239667_contigs.region001',
 'c00080_NODE_80...region001': 'ERR2239734_contigs.region001',
 'c00181_NODE_18...region001': 'ERR2239562_contigs.region002',
 'c00213_NODE_21...region001': 'ERR2239446_contigs.region002',
 'c00221_NODE_22...region001': 'ERR2239756_contigs.region002',
 'c00252_NODE_25...region001': 'ERR3946881_contigs.region003',
 'c00867_NODE_86...region001': 'ERR2239236_contigs.region005',
 'c01847_NODE_18...region001': 'ERR3941051_contigs.region011',
 'c02317_NODE_23...region001': 'ERR3941051_contigs.region012',
 'c04387_NODE_43...region001': 'ERR3941051_contigs.regi

In [7]:
def get_feature(updated_name,name_dict):
    for key in name_dict.keys():
        if name_dict[key] == updated_name:
            input_handle = open("/Volumes/TFL210426/ming_output_round4/antismash/%s.fasta_output_results/%s.gbk"%(updated_name.split('.')[0],key),'r')
            for seq_record in SeqIO.parse(input_handle,'genbank'):
                edge_list,type_list = [],[]
                for feature in seq_record.features:
                    if feature.type == 'cand_cluster':
                        for qual in feature.qualifiers:
                            if qual == 'contig_edge':
                                ctg_edge = feature.qualifiers[qual]
                                edge_list.append(ctg_edge[0])
                            if qual == 'product':
                                bgc_type = feature.qualifiers[qual]
                                for item_type in bgc_type:
                                    if item_type not in type_list:
                                        type_list.append(item_type)
                return edge_list,type_list
                
get_feature('GCA_001767235.region004',names_dict)

(['False'], ['NRPS'])

## Finding out what are all antiSMASH classes to create classes_dict

In [8]:
def get_feature(updated_name,name_dict):
    for key in name_dict.keys():
        if name_dict[key] == updated_name:
            filename = "/Volumes/TFL210426/ming_output_round4/antismash/%s.fasta_output_results/%s.gbk"%(updated_name.split('.')[0],key)
            if os.path.exists(filename):
                input_handle = open(filename,'r')
                for seq_record in SeqIO.parse(input_handle,'genbank'):
                    edge_list,type_list = [],[]
                    for feature in seq_record.features:
                        if feature.type == 'cand_cluster':
                            for qual in feature.qualifiers:
                                if qual == 'contig_edge':
                                    ctg_edge = feature.qualifiers[qual]
                                    edge_list.append(ctg_edge[0])
                                if qual == 'product':
                                    bgc_type = feature.qualifiers[qual]
                                    for item_type in bgc_type:
                                        if item_type not in type_list:
                                            type_list.append(item_type)
                    return edge_list,type_list
            else:
                return 'NA','NA'

In [9]:
all_classes = []

for i,r in training_df.iterrows():
    ctg_edge,class_list = get_feature(training_df['bgc'].loc[i],names_dict)
    for class_type in class_list:
        all_classes.append(class_type)
    
all_classes = np.unique(all_classes)

In [10]:
all_classes

array(['A', 'CDPS', 'LAP', 'N', 'NAGGN', 'NRPS', 'NRPS-like', 'PBDE',
       'PKS-like', 'PUFA', 'PpyS-KS', 'T1PKS', 'T2PKS', 'T3PKS',
       'TfuA-related', 'amglyccycl', 'arylpolyene', 'bacteriocin',
       'betalactone', 'blactam', 'butyrolactone', 'cyanobactin',
       'ectoine', 'furan', 'fused', 'hglE-KS', 'hserlactone', 'indole',
       'ladderane', 'lanthipeptide', 'lassopeptide', 'linaridin',
       'melanin', 'microviridin', 'nucleoside', 'oligosaccharide',
       'other', 'phenazine', 'phosphonate', 'proteusin', 'resorcinol',
       'sactipeptide', 'siderophore', 'terpene', 'thiopeptide',
       'transAT-PKS', 'transAT-PKS-like'], dtype='<U16')

In [11]:
len(all_classes)

47

## Creating a common dictionary

In [12]:
common_dict = {"A":"NA",
"CDPS":"NRPS",
"LAP":"RiPP",
"N":"NA",
"NAGGN":"NRPS",
"NRPS":"NRPS",
"NRPS-like":"NRPS",
"PBDE":"Phenolic",
"PKS-like":"PKS",
"PUFA":"PKS",
"PpyS-KS":"PKS",
"T1PKS":"PKS",
"T2PKS":"PKS",
"T3PKS":"PKS",
"TfuA-related":"RiPP",
"amglyccycl":"Minor",
"arylpolyene":["PKS","Phenolic"],
"bacteriocin":"RiPP",
"betalactone":"Minor",
"blactam":"Minor",
"butyrolactone":"Minor",
"cyanobactin":"RiPP",
"ectoine":"Minor",
"furan":"Minor",
"fused":"Minor",
"hglE-KS":"PKS",
"hserlactone":["PKS","NRPS"],
"indole":"Minor",
"ladderane":"RiPP",
"lanthipeptide":"RiPP",
"lassopeptide":"RiPP",
"linaridin":"RiPP",
"melanin":"NRPS",
"microviridin":"RiPP",
"nucleoside":"Nucleoside",
"oligosaccharide":"Oligosaccharide",
"other":"Other",
"phenazine":"Minor",
"phosphonate":"Phosphonate",
"proteusin":"RiPP",
"resorcinol":"PKS",
"sactipeptide":"RiPP",
"siderophore":"Siderophore",
"terpene":"Terpene",
"thiopeptide":"RiPP",
"transAT-PKS":"PKS",
"transAT-PKS-like":"PKS"}

In [13]:
common_dict

{'A': 'NA',
 'CDPS': 'NRPS',
 'LAP': 'RiPP',
 'N': 'NA',
 'NAGGN': 'NRPS',
 'NRPS': 'NRPS',
 'NRPS-like': 'NRPS',
 'PBDE': 'Phenolic',
 'PKS-like': 'PKS',
 'PUFA': 'PKS',
 'PpyS-KS': 'PKS',
 'T1PKS': 'PKS',
 'T2PKS': 'PKS',
 'T3PKS': 'PKS',
 'TfuA-related': 'RiPP',
 'amglyccycl': 'Minor',
 'arylpolyene': ['PKS', 'Phenolic'],
 'bacteriocin': 'RiPP',
 'betalactone': 'Minor',
 'blactam': 'Minor',
 'butyrolactone': 'Minor',
 'cyanobactin': 'RiPP',
 'ectoine': 'Minor',
 'furan': 'Minor',
 'fused': 'Minor',
 'hglE-KS': 'PKS',
 'hserlactone': ['PKS', 'NRPS'],
 'indole': 'Minor',
 'ladderane': 'RiPP',
 'lanthipeptide': 'RiPP',
 'lassopeptide': 'RiPP',
 'linaridin': 'RiPP',
 'melanin': 'NRPS',
 'microviridin': 'RiPP',
 'nucleoside': 'Nucleoside',
 'oligosaccharide': 'Oligosaccharide',
 'other': 'Other',
 'phenazine': 'Minor',
 'phosphonate': 'Phosphonate',
 'proteusin': 'RiPP',
 'resorcinol': 'PKS',
 'sactipeptide': 'RiPP',
 'siderophore': 'Siderophore',
 'terpene': 'Terpene',
 'thiopeptide': '

## Expanding training dataframe

In [14]:
training_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label,bgc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000737335.region031
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994635.region018
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000170895.region002
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994615.region022
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF2,DS570624.region002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF995,ERR3946851_contigs.region013
5417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3941157_contigs.region006
5418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3946770_contigs.region007
5419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF997,ERR3946819_contigs.region008


In [15]:
testing_df = pd.read_csv("./outputs/testing_df-NPOmixv1.0-TFL210811.tsv",index_col=0,sep='\t')

testing_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspCM6v1,SspCM7v1,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1
CCMSLIB00004679300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00004681487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00004681486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00004679299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00004681481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00000001706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00004679298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00000479759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00005724004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCMSLIB00000840594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
networked_cols = []

for item in testing_df.columns:
    if item in training_df.columns:
        networked_cols.append(item)
        
len(networked_cols)

1040

In [17]:
filt_training_df = training_df[networked_cols]
label_col = training_df['label']
filt_training_df = filt_training_df[(filt_training_df.T != 0).any()]
filt_label_col = []
for n,item in enumerate(label_col):
    if n in filt_training_df.index:
        filt_label_col.append(item)
filt_training_df['label'] = filt_label_col
filt_training_df = filt_training_df.sort_index(axis=1)
training_bgcs = []
count = 0
for item in training_bgcs:
    if count in filt_training_df.index:
        training_bgcs.append(item)
    count += 1
    
filt_indexes = []
filt_indexes = filt_training_df.index
filt_training_df = filt_training_df.reset_index(drop=True)

filt_training_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspCM7v1,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
4,0.0,0.0,0.0,0.920719,0.0,0.0,0.0,0.0,0.0,0.984742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF995
4757,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996
4758,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996
4759,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF997


In [18]:
final_classes = []
combined_list = []

for i,r in training_df.iterrows():
    ctg_edge,class_list = get_feature(training_df['bgc'].loc[i],names_dict)
    combined_classes = []
    for bgc_class in class_list:
        if type(common_dict[bgc_class]) != list:
            combined_classes.append(common_dict[bgc_class])
            if common_dict[bgc_class] not in final_classes:
                final_classes.append(common_dict[bgc_class])
        else:
            for item in common_dict[bgc_class]:
                combined_classes.append(item)
                if item not in final_classes:
                    final_classes.append(item)
    combined_list.append(combined_classes)
    print(i,class_list,'-',combined_classes)

0 ['siderophore'] - ['Siderophore']
1 ['siderophore'] - ['Siderophore']
2 ['siderophore'] - ['Siderophore']
3 ['siderophore'] - ['Siderophore']
4 NA - ['NA', 'NA']
5 NA - ['NA', 'NA']
6 ['siderophore'] - ['Siderophore']
7 ['siderophore'] - ['Siderophore']
8 ['siderophore'] - ['Siderophore']
9 ['siderophore'] - ['Siderophore']
10 ['siderophore'] - ['Siderophore']
11 ['siderophore'] - ['Siderophore']
12 ['terpene'] - ['Terpene']
13 ['siderophore'] - ['Siderophore']
14 ['siderophore'] - ['Siderophore']
15 NA - ['NA', 'NA']
16 ['siderophore'] - ['Siderophore']
17 ['siderophore'] - ['Siderophore']
18 ['siderophore'] - ['Siderophore']
19 ['siderophore'] - ['Siderophore']
20 ['siderophore'] - ['Siderophore']
21 ['siderophore'] - ['Siderophore']
22 ['siderophore'] - ['Siderophore']
23 ['siderophore'] - ['Siderophore']
24 ['siderophore'] - ['Siderophore']
25 ['siderophore'] - ['Siderophore']
26 ['T1PKS', 'NRPS'] - ['PKS', 'NRPS']
27 ['siderophore'] - ['Siderophore']
28 ['siderophore'] - ['Sider

260 ['bacteriocin'] - ['RiPP']
261 ['T1PKS', 'NRPS'] - ['PKS', 'NRPS']
262 ['T1PKS', 'NRPS'] - ['PKS', 'NRPS']
263 ['bacteriocin'] - ['RiPP']
264 ['NRPS', 'blactam', 'T1PKS'] - ['NRPS', 'Minor', 'PKS']
265 ['NRPS', 'T1PKS'] - ['NRPS', 'PKS']
266 NA - ['NA', 'NA']
267 ['T1PKS', 'NRPS'] - ['PKS', 'NRPS']
268 ['NRPS', 'T1PKS'] - ['NRPS', 'PKS']
269 ['bacteriocin'] - ['RiPP']
270 ['transAT-PKS', 'PKS-like', 'bacteriocin', 'T1PKS', 'NRPS'] - ['PKS', 'PKS', 'RiPP', 'PKS', 'NRPS']
271 ['NRPS', 'T1PKS'] - ['NRPS', 'PKS']
272 ['T1PKS', 'NRPS'] - ['PKS', 'NRPS']
273 ['T1PKS', 'NRPS'] - ['PKS', 'NRPS']
274 ['T1PKS', 'NRPS'] - ['PKS', 'NRPS']
275 ['NRPS', 'T1PKS'] - ['NRPS', 'PKS']
276 ['NRPS'] - ['NRPS']
277 ['terpene', 'NRPS-like', 'betalactone'] - ['Terpene', 'NRPS', 'Minor']
278 ['terpene', 'NRPS-like', 'betalactone'] - ['Terpene', 'NRPS', 'Minor']
279 ['terpene'] - ['Terpene']
280 ['terpene'] - ['Terpene']
281 ['terpene'] - ['Terpene']
282 ['NRPS'] - ['NRPS']
283 ['lanthipeptide'] - ['RiPP']


464 ['arylpolyene'] - ['PKS', 'Phenolic']
465 ['arylpolyene'] - ['PKS', 'Phenolic']
466 ['arylpolyene'] - ['PKS', 'Phenolic']
467 ['arylpolyene'] - ['PKS', 'Phenolic']
468 ['arylpolyene'] - ['PKS', 'Phenolic']
469 ['arylpolyene'] - ['PKS', 'Phenolic']
470 ['arylpolyene'] - ['PKS', 'Phenolic']
471 ['arylpolyene'] - ['PKS', 'Phenolic']
472 ['arylpolyene'] - ['PKS', 'Phenolic']
473 ['arylpolyene'] - ['PKS', 'Phenolic']
474 ['arylpolyene'] - ['PKS', 'Phenolic']
475 ['arylpolyene'] - ['PKS', 'Phenolic']
476 ['arylpolyene'] - ['PKS', 'Phenolic']
477 ['arylpolyene'] - ['PKS', 'Phenolic']
478 ['arylpolyene'] - ['PKS', 'Phenolic']
479 ['arylpolyene'] - ['PKS', 'Phenolic']
480 ['arylpolyene'] - ['PKS', 'Phenolic']
481 ['arylpolyene'] - ['PKS', 'Phenolic']
482 ['arylpolyene'] - ['PKS', 'Phenolic']
483 ['arylpolyene'] - ['PKS', 'Phenolic']
484 ['arylpolyene'] - ['PKS', 'Phenolic']
485 ['arylpolyene'] - ['PKS', 'Phenolic']
486 ['arylpolyene'] - ['PKS', 'Phenolic']
487 ['arylpolyene'] - ['PKS', 'Phe

674 ['NRPS', 'phenazine'] - ['NRPS', 'Minor']
675 ['phenazine'] - ['Minor']
676 ['phenazine'] - ['Minor']
677 ['phenazine'] - ['Minor']
678 ['phenazine'] - ['Minor']
679 ['phenazine'] - ['Minor']
680 ['phenazine'] - ['Minor']
681 ['phenazine'] - ['Minor']
682 ['phenazine'] - ['Minor']
683 ['phenazine'] - ['Minor']
684 ['NRPS', 'phenazine'] - ['NRPS', 'Minor']
685 ['NRPS', 'phenazine'] - ['NRPS', 'Minor']
686 ['NRPS', 'phenazine'] - ['NRPS', 'Minor']
687 ['NRPS', 'phenazine'] - ['NRPS', 'Minor']
688 ['NRPS-like'] - ['NRPS']
689 ['NRPS-like'] - ['NRPS']
690 ['NRPS-like'] - ['NRPS']
691 ['NRPS-like'] - ['NRPS']
692 ['NRPS-like'] - ['NRPS']
693 ['NRPS-like'] - ['NRPS']
694 ['NRPS-like'] - ['NRPS']
695 ['NRPS-like'] - ['NRPS']
696 ['NRPS-like'] - ['NRPS']
697 ['NRPS-like'] - ['NRPS']
698 ['NRPS-like'] - ['NRPS']
699 ['NRPS-like'] - ['NRPS']
700 ['NRPS-like'] - ['NRPS']
701 ['hserlactone'] - ['PKS', 'NRPS']
702 ['hserlactone'] - ['PKS', 'NRPS']
703 ['hserlactone'] - ['PKS', 'NRPS']
704 ['hse

936 ['bacteriocin'] - ['RiPP']
937 ['bacteriocin'] - ['RiPP']
938 ['bacteriocin'] - ['RiPP']
939 ['bacteriocin'] - ['RiPP']
940 ['bacteriocin'] - ['RiPP']
941 ['bacteriocin'] - ['RiPP']
942 ['bacteriocin'] - ['RiPP']
943 ['bacteriocin'] - ['RiPP']
944 ['bacteriocin'] - ['RiPP']
945 ['NRPS'] - ['NRPS']
946 ['NRPS'] - ['NRPS']
947 ['NRPS'] - ['NRPS']
948 ['NRPS'] - ['NRPS']
949 ['NRPS'] - ['NRPS']
950 ['NRPS'] - ['NRPS']
951 ['NRPS'] - ['NRPS']
952 ['NRPS'] - ['NRPS']
953 ['NRPS'] - ['NRPS']
954 ['NRPS'] - ['NRPS']
955 ['NRPS'] - ['NRPS']
956 ['NRPS'] - ['NRPS']
957 ['NRPS'] - ['NRPS']
958 ['NRPS'] - ['NRPS']
959 ['NRPS'] - ['NRPS']
960 ['NRPS'] - ['NRPS']
961 ['NRPS'] - ['NRPS']
962 ['NRPS'] - ['NRPS']
963 ['NRPS'] - ['NRPS']
964 ['NRPS'] - ['NRPS']
965 ['NRPS'] - ['NRPS']
966 ['NRPS'] - ['NRPS']
967 ['NRPS'] - ['NRPS']
968 ['NRPS'] - ['NRPS']
969 ['NRPS'] - ['NRPS']
970 ['NRPS'] - ['NRPS']
971 ['NRPS'] - ['NRPS']
972 ['NRPS'] - ['NRPS']
973 ['NRPS'] - ['NRPS']
974 ['NRPS'] - ['NRPS']
9

...

4176 ['sactipeptide'] - ['RiPP']
4177 ['NRPS-like'] - ['NRPS']
4178 ['bacteriocin'] - ['RiPP']
4179 ['sactipeptide'] - ['RiPP']
4180 ['sactipeptide'] - ['RiPP']
4181 ['sactipeptide'] - ['RiPP']
4182 ['sactipeptide'] - ['RiPP']
4183 ['sactipeptide'] - ['RiPP']
4184 ['sactipeptide'] - ['RiPP']
4185 ['sactipeptide'] - ['RiPP']
4186 ['sactipeptide'] - ['RiPP']
4187 ['NRPS'] - ['NRPS']
4188 ['sactipeptide'] - ['RiPP']
4189 ['sactipeptide'] - ['RiPP']
4190 ['sactipeptide'] - ['RiPP']
4191 ['sactipeptide'] - ['RiPP']
4192 ['sactipeptide'] - ['RiPP']
4193 ['sactipeptide'] - ['RiPP']
4194 ['sactipeptide'] - ['RiPP']
4195 ['sactipeptide'] - ['RiPP']
4196 ['sactipeptide'] - ['RiPP']
4197 ['sactipeptide'] - ['RiPP']
4198 ['sactipeptide'] - ['RiPP']
4199 ['sactipeptide'] - ['RiPP']
4200 ['sactipeptide'] - ['RiPP']
4201 ['lassopeptide'] - ['RiPP']
4202 ['sactipeptide'] - ['RiPP']
4203 ['sactipeptide'] - ['RiPP']
4204 ['sactipeptide'] - ['RiPP']
4205 ['sactipeptide'] - ['RiPP']
4206 ['sactipeptide'] 

4441 ['NRPS-like'] - ['NRPS']
4442 ['NRPS-like'] - ['NRPS']
4443 ['lassopeptide'] - ['RiPP']
4444 ['NRPS-like'] - ['NRPS']
4445 ['NRPS-like'] - ['NRPS']
4446 ['NRPS-like'] - ['NRPS']
4447 ['NRPS-like'] - ['NRPS']
4448 ['NRPS-like'] - ['NRPS']
4449 ['NRPS-like'] - ['NRPS']
4450 ['NRPS-like'] - ['NRPS']
4451 ['NRPS-like'] - ['NRPS']
4452 ['arylpolyene'] - ['PKS', 'Phenolic']
4453 ['NRPS-like'] - ['NRPS']
4454 ['NRPS-like'] - ['NRPS']
4455 ['NRPS-like'] - ['NRPS']
4456 ['NRPS-like'] - ['NRPS']
4457 ['NRPS-like'] - ['NRPS']
4458 ['NRPS'] - ['NRPS']
4459 ['NRPS-like'] - ['NRPS']
4460 ['NRPS-like'] - ['NRPS']
4461 ['NRPS'] - ['NRPS']
4462 ['NRPS-like'] - ['NRPS']
4463 ['NRPS-like'] - ['NRPS']
4464 ['NRPS-like'] - ['NRPS']
4465 ['NRPS-like'] - ['NRPS']
4466 ['NRPS-like'] - ['NRPS']
4467 ['NRPS-like'] - ['NRPS']
4468 ['NRPS-like'] - ['NRPS']
4469 ['NRPS-like'] - ['NRPS']
4470 ['NRPS-like'] - ['NRPS']
4471 ['NRPS-like'] - ['NRPS']
4472 ['NRPS-like'] - ['NRPS']
4473 ['NRPS-like'] - ['NRPS']
4474

4701 ['NRPS-like'] - ['NRPS']
4702 ['arylpolyene'] - ['PKS', 'Phenolic']
4703 ['lassopeptide'] - ['RiPP']
4704 ['lassopeptide'] - ['RiPP']
4705 ['bacteriocin'] - ['RiPP']
4706 ['arylpolyene'] - ['PKS', 'Phenolic']
4707 ['NRPS'] - ['NRPS']
4708 ['NRPS-like'] - ['NRPS']
4709 ['NRPS-like'] - ['NRPS']
4710 ['NRPS'] - ['NRPS']
4711 ['NRPS'] - ['NRPS']
4712 ['arylpolyene'] - ['PKS', 'Phenolic']
4713 ['hserlactone'] - ['PKS', 'NRPS']
4714 ['hserlactone'] - ['PKS', 'NRPS']
4715 ['bacteriocin'] - ['RiPP']
4716 ['bacteriocin'] - ['RiPP']
4717 ['arylpolyene'] - ['PKS', 'Phenolic']
4718 ['arylpolyene'] - ['PKS', 'Phenolic']
4719 ['arylpolyene'] - ['PKS', 'Phenolic']
4720 ['bacteriocin'] - ['RiPP']
4721 ['sactipeptide'] - ['RiPP']
4722 ['LAP'] - ['RiPP']
4723 ['NRPS-like'] - ['NRPS']
4724 ['NRPS'] - ['NRPS']
4725 ['sactipeptide'] - ['RiPP']
4726 ['sactipeptide'] - ['RiPP']
4727 ['sactipeptide'] - ['RiPP']
4728 ['sactipeptide'] - ['RiPP']
4729 ['bacteriocin'] - ['RiPP']
4730 ['NRPS-like'] - ['NRPS']

4953 ['NRPS-like'] - ['NRPS']
4954 ['NRPS-like'] - ['NRPS']
4955 ['NRPS-like'] - ['NRPS']
4956 ['NRPS'] - ['NRPS']
4957 ['arylpolyene'] - ['PKS', 'Phenolic']
4958 ['NRPS'] - ['NRPS']
4959 ['resorcinol'] - ['PKS']
4960 ['resorcinol'] - ['PKS']
4961 ['resorcinol'] - ['PKS']
4962 ['arylpolyene'] - ['PKS', 'Phenolic']
4963 ['arylpolyene'] - ['PKS', 'Phenolic']
4964 ['arylpolyene'] - ['PKS', 'Phenolic']
4965 ['NRPS'] - ['NRPS']
4966 ['NRPS'] - ['NRPS']
4967 ['NRPS-like'] - ['NRPS']
4968 ['NRPS'] - ['NRPS']
4969 ['NRPS'] - ['NRPS']
4970 ['NRPS-like'] - ['NRPS']
4971 ['arylpolyene'] - ['PKS', 'Phenolic']
4972 ['arylpolyene'] - ['PKS', 'Phenolic']
4973 ['LAP'] - ['RiPP']
4974 ['lanthipeptide'] - ['RiPP']
4975 ['LAP'] - ['RiPP']
4976 ['NRPS'] - ['NRPS']
4977 ['NRPS'] - ['NRPS']
4978 ['NRPS'] - ['NRPS']
4979 ['arylpolyene'] - ['PKS', 'Phenolic']
4980 ['NRPS-like'] - ['NRPS']
4981 ['NRPS'] - ['NRPS']
4982 ['NRPS'] - ['NRPS']
4983 ['NRPS'] - ['NRPS']
4984 ['NRPS'] - ['NRPS']
4985 ['arylpolyene'] -

5209 ['NRPS'] - ['NRPS']
5210 ['NRPS'] - ['NRPS']
5211 ['NRPS-like'] - ['NRPS']
5212 ['NRPS'] - ['NRPS']
5213 ['NRPS-like'] - ['NRPS']
5214 ['NRPS-like'] - ['NRPS']
5215 ['NRPS-like'] - ['NRPS']
5216 ['arylpolyene'] - ['PKS', 'Phenolic']
5217 ['arylpolyene'] - ['PKS', 'Phenolic']
5218 ['resorcinol'] - ['PKS']
5219 ['resorcinol'] - ['PKS']
5220 ['resorcinol'] - ['PKS']
5221 ['resorcinol'] - ['PKS']
5222 ['lanthipeptide'] - ['RiPP']
5223 ['lanthipeptide'] - ['RiPP']
5224 ['lanthipeptide'] - ['RiPP']
5225 ['NRPS'] - ['NRPS']
5226 ['NRPS'] - ['NRPS']
5227 ['siderophore'] - ['Siderophore']
5228 ['siderophore'] - ['Siderophore']
5229 ['arylpolyene'] - ['PKS', 'Phenolic']
5230 ['arylpolyene'] - ['PKS', 'Phenolic']
5231 ['arylpolyene'] - ['PKS', 'Phenolic']
5232 ['arylpolyene'] - ['PKS', 'Phenolic']
5233 ['arylpolyene'] - ['PKS', 'Phenolic']
5234 ['arylpolyene'] - ['PKS', 'Phenolic']
5235 ['lanthipeptide'] - ['RiPP']
5236 ['lanthipeptide'] - ['RiPP']
5237 ['T3PKS'] - ['PKS']
5238 ['T3PKS'] - [

In [19]:
final_classes

['Siderophore',
 'NA',
 'Terpene',
 'PKS',
 'NRPS',
 'Minor',
 'RiPP',
 'Phosphonate',
 'Other',
 'Oligosaccharide',
 'Phenolic',
 'Nucleoside']

In [20]:
ext_train_df = pd.DataFrame(columns=(final_classes))

for i,combined_classes in enumerate(combined_list):
    row_extension = []
    for final_class in final_classes:
        if final_class in combined_classes:
            row_extension.append(1)
        else:
            row_extension.append(0)
    ext_train_df.loc[i] = row_extension

In [21]:
ext_train_df = ext_train_df.loc[filt_indexes]
ext_train_df = ext_train_df.reset_index(drop=True)

ext_train_df

,Siderophore,NA,Terpene,PKS,NRPS,Minor,RiPP,Phosphonate,Other,Oligosaccharide,Phenolic,Nucleoside
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4756,0,0,0,0,1,0,0,0,0,0,0,0
4757,0,0,0,0,1,0,0,0,0,0,0,0
4758,0,0,0,0,1,0,0,0,0,0,0,0
4759,0,0,0,0,1,0,0,0,0,0,0,0


In [22]:
merged_train_df = filt_training_df.join(ext_train_df, how='outer')

merged_train_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,Terpene,PKS,NRPS,Minor,RiPP,Phosphonate,Other,Oligosaccharide,Phenolic,Nucleoside
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.920719,0.0,0.0,0.0,0.0,0.0,0.984742,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0
4757,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0
4758,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0
4759,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0


## Obtaining final training dataframe

In [23]:
training_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label,bgc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000737335.region031
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994635.region018
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_000170895.region002
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1,GCA_002994615.region022
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF2,DS570624.region002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF995,ERR3946851_contigs.region013
5417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3941157_contigs.region006
5418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996,ERR3946770_contigs.region007
5419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF997,ERR3946819_contigs.region008


In [24]:
final_training_df = merged_train_df[networked_cols+final_classes]
label_col = merged_train_df['label']
final_training_df = final_training_df[(final_training_df.T != 0).any()]

filt_label_col = []
for n,item in enumerate(label_col):
    if n in final_training_df.index:
        filt_label_col.append(item)


final_training_df['label'] = filt_label_col
final_training_df = final_training_df.sort_index(axis=1)

final_training_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Terpene,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF1
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF1
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF1
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF1
4,0.0,0.0,0.0,0.920719,0.0,0.0,0.0,0.0,0.0,0.984742,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF995
4757,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF996
4758,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF996
4759,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,GCF997


In [25]:
final_training_df.shape

(4761, 1053)

## Expanding testing dataframe

In [26]:
test_ext_df = pd.read_csv('/Users/tiagoferreiraleao/Downloads/KNN_apporach/common_dict_test_extended.csv',index_col=0)

test_ext_df

,Siderophore,NA,Terpene,PKS,NRPS,Minor,RiPP,Phosphonate,Other,Oligosaccharide,Phenolic,Nucleoside
CCMSLIB00004679300,0,0,0,0,1,0,0,0,0,0,0,0
CCMSLIB00004681487,0,0,0,1,1,0,0,0,0,0,0,0
CCMSLIB00004681486,0,0,0,1,1,0,0,0,0,0,0,0
CCMSLIB00004679299,0,0,0,0,1,0,0,0,0,0,0,0
CCMSLIB00004681481,0,0,0,1,1,0,0,0,0,0,0,0
CCMSLIB00004679298,0,0,0,0,1,0,0,0,0,0,0,0
CCMSLIB00000479759,0,0,0,1,0,0,0,0,0,0,0,0
CCMSLIB00000840594,0,0,0,0,1,0,0,0,0,0,0,0
CCMSLIB00004681475,0,0,0,1,1,0,0,0,0,0,0,0
CCMSLIB00000579285,0,0,0,1,1,0,0,0,0,0,0,0


In [27]:
final_testing_df = testing_df.join(test_ext_df, how='outer')
final_testing_df = final_testing_df.sort_index(axis=1)

final_testing_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspCM7v1,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Terpene,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1
CCMSLIB00000001553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00000001706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00000001708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00000001751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00000479759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00000579285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00000840594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00004679298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00004679299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
CCMSLIB00004679300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [28]:
final_testing_df.shape

(15, 1052)

## Running KNN

In [29]:
affinity_df = pd.read_csv('./outputs/affinity_df-round4-TFL210427.txt',index_col=0)

affinity_df

,2515154094,2515154124,2515154177,2515154206,2516143022,2516493032,2517434008,2517572145,2517572157,2517572165,...,SspCM7v1,SspOBRC6v1,SspSR1v1,Stre_inte_F0413_V1,Stre_para_CC87K_V1,Stre_sang_CC94A_V1,Trep_dent_ASLM_V1,Trep_dent_US-Trep_V1,VspACP1v1,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF995
5677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996
5678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF996
5679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCF997


In [30]:
X_div = final_training_df.drop("label", axis=1)
X_div.shape

(4761, 1052)

In [31]:
y_div = final_training_df["label"]
y_div.shape

(4761,)

In [32]:
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(X_div,y_div)

distances, indices = nbrs.kneighbors(final_testing_df)

In [33]:
y_div = y_div.reset_index(drop=True)

neighbors_array = []

for item in indices:
    candidate_list = []
    for i in range(3):
        candidate_list.append(y_div[item[i]])
    neighbors_array.append(candidate_list)

neighbors_array = np.asarray(neighbors_array)
        
neighbors_array

array([['GCF471', 'GCF498', 'GCF550'],
       ['GCF471', 'GCF498', 'GCF550'],
       ['GCF471', 'GCF498', 'GCF550'],
       ['GCF471', 'GCF498', 'GCF550'],
       ['GCF117', 'GCF126', 'GCF120'],
       ['GCF476', 'GCF219', 'GCF235'],
       ['GCF740', 'GCF740', 'GCF739'],
       ['GCF450', 'GCF360', 'GCF360'],
       ['GCF450', 'GCF360', 'GCF360'],
       ['GCF450', 'GCF360', 'GCF360'],
       ['GCF476', 'GCF219', 'GCF235'],
       ['GCF476', 'GCF219', 'GCF235'],
       ['GCF476', 'GCF219', 'GCF235'],
       ['GCF476', 'GCF219', 'GCF235'],
       ['GCF471', 'GCF498', 'GCF550']], dtype='<U6')

In [34]:
training_bgcs = pickle.load( open("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/NPOmix/outputs/affinity_bgcs-round4-TFL210427.txt", "rb" ) )

training_bgcs

['GCA_000737335.region031',
 'GCA_002994635.region018',
 'GCA_000170895.region002',
 'GCA_002994615.region022',
 'BGC0000844.1',
 'DS570624.region002',
 'BGC0000845.1',
 'BGC0000319.1',
 'DS570624.region019',
 'BGC0000373.1',
 'BGC0000940.1',
 '2747842505.region030',
 'GCA_002899455.region014',
 '2517572165.region052',
 'GCA_000365385.region030',
 'GCA_000377145.region006',
 'GCA_000424965.region016',
 'GCA_000377525.region018',
 'GCA_000514715.region025',
 '651717009.region006',
 'DS570624.region031',
 'GCA_000426165.region008',
 '2518285564.region046',
 'GCA_000158915.region019',
 'GCA_000377165.region004',
 'GCA_000377125.region004',
 '2515154206.region008',
 'GCA_000739105.region018',
 'GCA_000527195.region007',
 'BGC0000941.1',
 'GCA_000156695.region008',
 'GCA_000424765.region022',
 'GCA_000365385.region010',
 'GCA_000424805.region006',
 'GCA_000240165.region027',
 'GCA_000515055.region002',
 'GCA_000156435.region015',
 'BGC0001478.1',
 'GCA_000203835.region011',
 'GCA_000701285.

In [35]:
mibig_gnps_dict = {}
mibig_gnps_df = pd.read_csv('/Users/tiagoferreiraleao/Dropbox/tiago-NAS/NPOmix/inputs/matched_mibig_gnps_update.tsv',sep='\t')
mibig_gnps_dict = dict(zip(mibig_gnps_df['# mgf_spectrum_id'],mibig_gnps_df['mibig_id']))

In [36]:
correct_count = 0
incorrect_count = 0
ccms_id_label_dict = {}

for index,ccms_id in enumerate(final_testing_df.index):
    if ccms_id in mibig_gnps_dict:
        for i,bgc in enumerate(training_bgcs):
            if mibig_gnps_dict[ccms_id] in bgc:
                print(ccms_id,affinity_df.label[i],neighbors_array[index])
                if affinity_df.label[i] in neighbors_array[index]:
                    print("Correct")
                    correct_count += 1
                    ccms_id_label_dict[ccms_id] = affinity_df.label[i]
                else:
                    print("Incorrect")
                    incorrect_count += 1
                    ccms_id_label_dict[ccms_id] = affinity_df.label[i]

total_count = incorrect_count + correct_count

print('Precision = %s/%s = %s'%(correct_count,total_count,(correct_count/total_count)))

CCMSLIB00000001706 GCF471 ['GCF471' 'GCF498' 'GCF550']
Correct
CCMSLIB00000001708 GCF471 ['GCF471' 'GCF498' 'GCF550']
Correct
CCMSLIB00000479759 GCF320 ['GCF117' 'GCF126' 'GCF120']
Incorrect
CCMSLIB00000579285 GCF476 ['GCF476' 'GCF219' 'GCF235']
Correct
CCMSLIB00000840594 GCF488 ['GCF740' 'GCF740' 'GCF739']
Incorrect
CCMSLIB00004679298 GCF450 ['GCF450' 'GCF360' 'GCF360']
Correct
CCMSLIB00004679299 GCF450 ['GCF450' 'GCF360' 'GCF360']
Correct
CCMSLIB00004679300 GCF450 ['GCF450' 'GCF360' 'GCF360']
Correct
CCMSLIB00004681475 GCF476 ['GCF476' 'GCF219' 'GCF235']
Correct
CCMSLIB00004681481 GCF476 ['GCF476' 'GCF219' 'GCF235']
Correct
CCMSLIB00004681486 GCF476 ['GCF476' 'GCF219' 'GCF235']
Correct
CCMSLIB00004681487 GCF476 ['GCF476' 'GCF219' 'GCF235']
Correct
CCMSLIB00005724004 GCF498 ['GCF471' 'GCF498' 'GCF550']
Correct
Precision = 11/13 = 0.8461538461538461


In [37]:
# top-1
10/15

0.6666666666666666

In [38]:
# top-3
11/15

0.7333333333333333